In [1]:
import calendar
import pandas as pd
from datetime import date, timedelta
from sqlalchemy import create_engine

engine = create_engine("sqlite:///c:\\ruby\\port_lite\\db\\development.sqlite3")
conlite = engine.connect()

engine = create_engine("mysql+pymysql://root:@localhost:3306/stock")
const = engine.connect()

pd.set_option('display.max_row',None)

data_path = "../data/"
csv_path = "\\Users\\User\\iCloudDrive\\"
box_path = "\\Users\\User\\Dropbox\\"
one_path = "\\Users\\User\\OneDrive\\Documents\\Data\\"

today = date.today()
yesterday = today - timedelta(days=1)
today, yesterday

(datetime.date(2022, 5, 15), datetime.date(2022, 5, 14))

In [2]:
today = today - timedelta(days=2)
wkago = today - timedelta(days=7)
#wkago = date(2020, 12, 30)
today, wkago

(datetime.date(2022, 5, 13), datetime.date(2022, 5, 6))

### Restart and Run All Cells

### Weekly process or when stataus changes.

In [3]:
sqlDel = """DELETE FROM sales"""
rp = conlite.execute(sqlDel)
rp.rowcount

1812

In [4]:
sql = """
SELECT name
FROM orders 
ORDER BY name
"""
df = pd.read_sql(sql, conlite)

names = df['name'].values.tolist()
in_p = ", ".join(map(lambda name: "'%s'" % name, names))
in_p

"'BBL', 'BCH', 'BDMS', 'BGC', 'DIF', 'DOHOME', 'HREIT', 'IP', 'IVL', 'JASIF', 'KBANK', 'KKP', 'MAKRO', 'NER', 'PTT', 'PTTGC', 'RATCH', 'STA', 'SYNEX', 'WHART'"

### Get past one year data

In [5]:
one_year_ago = yesterday - timedelta(days=365)
one_year_ago

sql = """
SELECT * 
FROM price 
WHERE date >= '%s' AND name IN (%s) 
ORDER BY name, date"""
sql = sql % (one_year_ago, in_p)
print(sql)


SELECT * 
FROM price 
WHERE date >= '2021-05-14' AND name IN ('BBL', 'BCH', 'BDMS', 'BGC', 'DIF', 'DOHOME', 'HREIT', 'IP', 'IVL', 'JASIF', 'KBANK', 'KKP', 'MAKRO', 'NER', 'PTT', 'PTTGC', 'RATCH', 'STA', 'SYNEX', 'WHART') 
ORDER BY name, date


In [6]:
df = pd.read_sql(sql, const)
df.shape

(4839, 7)

In [7]:
data_path = "../data/"
file_name = "Yearly-Price-by-Name.csv"
output_file = data_path + file_name

df.set_index("name", inplace=True)
df.to_csv(output_file, header=None)

In [8]:
sql = """
SELECT name,trade 
FROM orders 
WHERE name IN (%s)
ORDER BY name
"""
sql = sql % in_p
orders = pd.read_sql(sql, conlite)
orders.set_index(['name'],inplace=True)
orders

,trade
name,
BBL,B
BCH,S
BDMS,B
BGC,B
DIF,B
DOHOME,S
HREIT,B
IP,S
IVL,B


### Create monitors from stocks

In [9]:
sql = """
SELECT name, trade 
FROM orders 
ORDER BY name
"""
monitors = pd.read_sql(sql, conlite)
monitors.set_index(["name"], inplace=True)
monitors

,trade
name,
BBL,B
BCH,S
BDMS,B
BGC,B
DIF,B
DOHOME,S
HREIT,B
IP,S
IVL,B


In [10]:
monitors.shape[0]

20

In [11]:
file_name = "monitors.csv"
data_file = data_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name

orders.to_csv(data_file, header=None)
orders.to_csv(box_file)

In [12]:
sql = """
SELECT trade, COUNT(*) AS items 
FROM orders
GROUP BY trade
ORDER BY trade
"""
grp = pd.read_sql(sql, conlite)
grp

,trade,items
0,B,10
1,S,10


In [13]:
file_name = 'price-uploads.csv'
url = data_path + file_name
url

'../data/price-uploads.csv'

In [14]:
uploads = pd.read_csv(url)
uploads.sort_values(['name'],ascending=[True]).shape

(140, 1)

In [15]:
df_merge = pd.merge(orders, uploads, on='name', how='outer', indicator=True)
df_merge.sort_values(['name'],ascending=[True]).shape

(140, 3)

In [16]:
new_prices = df_merge['_merge'] == 'left_only'
df_merge[new_prices]

,name,trade,_merge


In [17]:
new_prices = df_merge['_merge'] == 'right_only'
df_merge[new_prices].shape

(120, 3)

In [18]:
sql = """
SELECT name, status, market
FROM stocks 
ORDER BY status, name
"""
stocks = pd.read_sql(sql, conlite)
stocks.set_index(["name"], inplace=True)
stocks

,status,market
name,,
DIF,B,SET
HREIT,B,SET
JASIF,B,SET
WHART,B,SET
BCH,I,SET100
DOHOME,I,SET100
IP,I,mai
MAKRO,I,SET
NER,I,SET


In [19]:
file_name = "stocks-all.csv"
data_file = data_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name
one_file = one_path + file_name

stocks.to_csv(data_file, header=None)
stocks.to_csv(output_file)
stocks.to_csv(box_file)
stocks.to_csv(one_file)

In [20]:
sql = """
SELECT name, status 
FROM stocks 
WHERE status IN ("B","O")
ORDER BY name
"""

buy_candidates = pd.read_sql(sql, conlite)
buy_candidates.set_index(["name"], inplace=True)
buy_candidates

,status
name,
AIT,O
BBL,O
BCPG,O
BDMS,O
BGC,O
BH,O
BPP,O
COTTO,O
DIF,B


In [21]:
buy_candidates.shape[0]

23

In [22]:
sql = """
SELECT name, status 
FROM stocks 
WHERE status IN ("I","S")
ORDER BY name
"""

sell_candidates = pd.read_sql(sql, conlite)
sell_candidates.set_index(["name"], inplace=True)
sell_candidates

,status
name,
BCH,I
DOHOME,I
IP,I
MAKRO,I
NER,I
PTT,I
PTTGC,S
RATCH,I
STA,S


In [23]:
sell_candidates.shape[0]

10